- [Paper: Optimal Online Adaptive Electric Vehicle Charging](http://netlab.caltech.edu/assets/publications/Guo-2017-OLP.pdf)
- [Pyomo - Optimization Modeling in Python](https://pyomo.readthedocs.io/en/stable/)
- [ TeX functions supported by KaTeX](https://katex.org/docs/supported.html)
- [TGC_LP: OLP in Excel for Enexis (OneDrive)](https://1drv.ms/x/s!AiogHeTeve1hj5NFFlBANUYBfoFAcA?e=Kj0bdQ)

# OLP: Online Linear Programming

<br>

<div style="border:1px solid blue; border-radius: 5px; background-color: #DDEDF9; color: black; padding: 0px">
    <div style="background-color: darkblue; color: white; padding: 10px; border-radius: 5px 5px 0 0">
        <strong>‼ Note:</strong>
    </div>
    <BR>
    >  The outcomes of the excel-model are the same as the pyomo-model !!! 
    <BR>
    >  Check: <strong>objective outcome</strong> and the <strong>Power to be supplied</strong> (Variables) 
    <BR>
    <BR>
</div>

### ✔ Pyomo-0

#### Model

This first model has only 1 EV/SE and has to determine the ``kW`` for the next ``n`` periods (n=5).  

The objective is to minimize the difference in the  ``theoretical maximum energy output [kWh]``, being ```Enexis max power [kW] * duration [h]``` and the ``actual delivered energy``, being ```actual power [kW] * duration [h]```.

By normalizing the duration, the resulting ``weighted-% energy delivered`` can easily be combined with any future objective like the ``weighted-% customer satisfaction``. 

This results in the following ``Objective function`` to be minimalized. 


##### Objective Function


$$
\begin{align*}
\text{minimize} \; &\sum_{j=1}^{n} w_{j} \left( 1 - \frac{x_{j}}{ex_{j}} \right), \\
\text{where} \; &w_{j} = \frac{pt_{j}}{\sum_{k=1}^{n} pt_{k}}, \; \forall j, k \in \{1, 2, \ldots, n\} \\
\end{align*}
$$

$ \text{Note: } j \text{ represents a specific time period out of } n \text{ time periods in the forecast horizon.}$ 

##### Decision Variables

The charging power at time slot $j$ are the decision variables, $x_{j}$ 

##### Constraints

The charging power to be used ($x_{j}$) in kW is subject to 4 constraints:

1. always needs to be positive, Vehicle to Grid (V2G) is **not** allowed.
1. less or equal than the Enexis maximum power output ($ex_j$) of Grid Connection. 
1. less or equal than the maximum power input ($ev_j$) of the EV onboard charger (obc)
1. less or equal than the maximum power output ($se_j$) of the EVSE.


$$
\begin{array}{llllll}
\text{s.t.} & x_{j} \geq 0  &  \forall j \in \{1, 2, \ldots, n\} \\
& x_{j} \leq ex_{j} & \forall j \in \{1, 2, \ldots, n\} \\
& x_{j} \leq ev_{j} & \forall j \in \{1, 2, \ldots, n\} \\
& x_{j} \leq se_{j} & \forall j \in \{1, 2, \ldots, n\} \\
\end{array} 
$$

$ \text{Note: } j \text{ represents a specific time period out of } n \text{ time periods in the forecast horizon.}$ 

This example relates to the ``pyomo_0`` sheet in this workbook om OneDrive: [TGC_LP](https://1drv.ms/x/s!AiogHeTeve1hj5NFFlBANUYBfoFAcA?e=Kj0bdQ)

#### Excel: results (sheet ``pyomo_0``) 

<img src="./images/pyomo_0.png" width="1000">

#### Code & Results

In [13]:
# --------------------------------------------------------------------------
# TGC: Tetris Game Charger
# --------------------------------------------------------------------------

from pyomo.environ import *
import numpy as np

# Create the weights w[j] for the parking 
pt = np.array([40, 10, 20, 10, 10])
w = pt/np.sum(pt)

# row reserved for future use
n = 5  # number of time periods in the future

EX_MPO = 14.72  # enexis max Power output for each time period (constant)
EV_MPI = 7.36  # EV max Power input for each time period
SE_MPO = 7.36  # EVSE max Power output for each time period


# --------------------------------------------------------------------------
# Abstract Model
# https://pyomo.readthedocs.io/en/stable/pyomo_overview/simple_examples.html#a-simple-abstract-pyomo-model
# --------------------------------------------------------------------------

model = AbstractModel()

# --------------------------------------------------------------------------
# Sets
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html
# --------------------------------------------------------------------------


model.J = RangeSet(1, n)  # set of time periods for a certain horizon (h=5)

# --------------------------------------------------------------------------
# Parameters
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Parameters.html
# --------------------------------------------------------------------------

# enexis max Power output for each time period (constant)
ex_kw_dv = {}  # deviations from the default value for enexis mpo per time period j
model.ex = Param(model.J, initialize=ex_kw_dv, default=EX_MPO)

# EV max Power input for each time period
ev_kw_dv = {}  # deviations from the default value for ev i and time period j
ev_kw_dv[3] = 5.00  # kW
ev_kw_dv[4] = 3.68  # kW
ev_kw_dv[5] = 0.00  # kW
#
#
#
#
#
#
model.ev = Param(model.J, initialize=ev_kw_dv, default=EV_MPI)

# EVSE max Power output for each time period
se_kw_dv = {}  # deviations from the default value for evse i and time period j
se_kw_dv[1] = 5.00  # kW
#
#
model.se = Param(model.J, initialize=se_kw_dv, default=SE_MPO)

# --------------------------------------------------------------------------
# Variables
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Variables.html
# --------------------------------------------------------------------------

model.x = Var(model.J, domain=NonNegativeReals)


# --------------------------------------------------------------------------
# Objective function
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Objectives.html
# --------------------------------------------------------------------------


def obj_expression(model):
    # return the expression for the objective
    return sum(w[j-1] * (1 - model.x[j]/model.ex[j]) for j in model.J)


model.OBJ = Objective(rule=obj_expression)

# --------------------------------------------------------------------------
# Constraints
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Expressions.html
# --------------------------------------------------------------------------


# is ev/se power <= enexis max power output for period j?
def demand_constraint_rule(model, j):
    return model.x[j] <= model.ex[j]


# is ev/se power <= ev max power input for period j?
def ev_mpi_constraint_rule(model, j):
    return model.x[j] <= model.ev[j]


# is ev/se power <= evse max power output for period j?
def se_mpo_constraint_rule(model, j):
    return model.x[j] <= model.se[j]


# the next lines create one constraint for each member of the set model.J
model.EX_MPO_Constraint = Constraint(model.J, rule=demand_constraint_rule)
model.EV_MPI_Constraint = Constraint(model.J, rule=ev_mpi_constraint_rule)
model.SE_MPO_Constraint = Constraint(model.J, rule=se_mpo_constraint_rule)


# --------------------------------------------------------------------------
# create a model instance and optimize
# https://pyomo.readthedocs.io/en/stable/working_abstractmodels/instantiating_models.html
# --------------------------------------------------------------------------

tgc = model.create_instance()

opt = pyomo.environ.SolverFactory("cplex")  # or 'mosek' or 'glpk'

opt.solve(tgc)

# --------------------------------------------------------------------------
# display solution
# --------------------------------------------------------------------------

# tgc.pprint()
print("\n")
print(f"Power to be supplied EVSE 1: {[round(tgc.x[x].value, 2) for x in range(1, 6)]}")


print("\nObjective Outcome = ", round(value(tgc.OBJ), 4))



Power to be supplied EVSE 1: [5.0, 7.36, 5.0, 3.68, 0.0]

Objective Outcome =  0.6902


### ✔ Pyomo-1

#### Model

Next, we extend the ``pyomo-0`` model by allowing multiple EV/SE parallel charging sessions, (m = 3).  
This results in the following ``Objective function`` to be minimized.

##### Objective Function


$$
\begin{align*}
\text{minimize} \; &\sum_{j=1}^{n} w_{j} \left( 1 - \frac{\sum_{i=1}^{m} x_{ij}}{ex_{j}} \right), \\
\text{where} \; &w_{j} = \frac{pt_{j}}{\sum_{k=1}^{n} pt_{k}}, \; \forall j, k \in \{1, 2, \ldots, n\}, \; \forall i \in \{1, 2, \ldots, m\} \\
\end{align*}
$$

$$
\begin{array}{rllll}
\text{Note: } & \\
& i \text{ represents a specific charge session EV/SE with } m \text{ being the maximum parallel charge sessions possible.} \\
& j \text{ represents a specific time period out of } n \text{ time periods in the forecast horizon.}
\end{array} 
$$


##### Decision Variables

The charging power for EV/SE $i$ at time slot $j$ are the decision variables, $x_{ij}$ 

##### Constraints

The charging power to be used ($ x_{ij} $) in kW is subject to 4 constraints:

1. always needs to be positive, Vehicle to Grid (V2G) is **not** allowed.
1. less or equal than the Enexis maximum power output ($ex_j$) of Grid Connection. 
1. less or equal than the maximum power input ($ev_j$) of the EV onboard charger (obc)
1. less or equal than the maximum power output ($se_j$) of the EVSE.


$$
\begin{array}{llll}
\text{s.t.} & x_{ij} \geq 0  &  \\
\sum_{i=1}^{m} & x_{ij} \leq ex_{j} & \\
& x_{ij} \leq ev_{ij} & \\
& x_{ij} \leq se_{ij} & \\ 
\end{array} 
$$



$$
\begin{array}{ll}
\forall i \in \{1, 2, \ldots, m,\} & \text{i =  charge session,} & \text{m = 3}  \\
\forall j \in \{1, 2, \ldots, n\} &\text{j =  time period,} & \text{n = 5}
\end{array}
$$

This example relates to the ``pyomo_1`` sheet in this workbook om OneDrive: [TGC_LP](https://1drv.ms/x/s!AiogHeTeve1hj5NFFlBANUYBfoFAcA?e=Kj0bdQ)

#### Excel: results (sheet ``pyomo_1``) 


<img src="./images/pyomo_1.png" width="1000">

#### Code & Results

In [9]:
# --------------------------------------------------------------------------
# TGC: Tetris Game Charger
# --------------------------------------------------------------------------

from pyomo.environ import *
import numpy as np

# Create the weights w[j] for the parking 
pt = np.array([40, 10, 20, 10, 10])
w = pt/np.sum(pt)

m = 3  # number of EVSEs
n = 5  # number of time periods in the future

EX_MPO = 14.72  # enexis max Power output for each time period (constant)
EV_MPI = 7.36  # EV max Power input for each time period
SE_MPO = 7.36  # EVSE max Power output for each time period


# --------------------------------------------------------------------------
# Abstract Model
# https://pyomo.readthedocs.io/en/stable/pyomo_overview/simple_examples.html#a-simple-abstract-pyomo-model
# --------------------------------------------------------------------------

model = AbstractModel()

# --------------------------------------------------------------------------
# Sets
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html
# --------------------------------------------------------------------------

model.I = RangeSet(1, m)  # set of EVSEs
model.J = RangeSet(1, n)  # set of time periods for a certain horizon (h=5)

# --------------------------------------------------------------------------
# Parameters
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Parameters.html
# --------------------------------------------------------------------------

# enexis max Power output for each time period (constant)
ex_kw_dv = {}  # deviations from the default value for enexis mpo per time period j
model.ex = Param(model.J, initialize=ex_kw_dv, default=EX_MPO)

# EV max Power input for each time period
ev_kw_dv = {}  # deviations from the default value for ev i and time period j
ev_kw_dv[1, 3] = 5.00  # kW
ev_kw_dv[1, 4] = 3.68  # kW
ev_kw_dv[1, 5] = 0.00  # kW
ev_kw_dv[2, 3] = 5.00  # kW
ev_kw_dv[2, 4] = 3.68  # kW
ev_kw_dv[2, 5] = 0.00  # kW
ev_kw_dv[3, 3] = 5.00  # kW
ev_kw_dv[3, 4] = 3.68  # kW
ev_kw_dv[3, 5] = 0.00  # kW
model.ev = Param(model.I, model.J, initialize=ev_kw_dv, default=EV_MPI)

# EVSE max Power output for each time period
se_kw_dv = {}  # deviations from the default value for evse i and time period j
se_kw_dv[1, 1] = 5.00  # kW
se_kw_dv[2, 1] = 5.00  # kW
se_kw_dv[3, 1] = 5.00  # kW
model.se = Param(model.I, model.J, initialize=se_kw_dv, default=SE_MPO)

# --------------------------------------------------------------------------
# Variables
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Variables.html
# --------------------------------------------------------------------------

model.x = Var(model.I, model.J, domain=NonNegativeReals)


# --------------------------------------------------------------------------
# Objective function
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Objectives.html
# --------------------------------------------------------------------------


def obj_expression(model):
    # return the expression for the objective
    return sum(w[j-1] * (1 - sum(model.x[i, j] for i in model.I)/model.ex[j]) 
               for j in model.J)
#     return sum(w[j-1] * (1 - model.x[j]/model.ex[j]) for j in model.J)

model.OBJ = Objective(rule=obj_expression)

# --------------------------------------------------------------------------
# Constraints
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Expressions.html
# --------------------------------------------------------------------------


# is ev/se power[i] <= enexis max power output for period j?
def demand_constraint_rule(model, j):
    return sum(model.x[i, j] for i in model.I) <= model.ex[j]


# is ev/se power[i] <= ev[i] max power input for period j?
def ev_mpi_constraint_rule(model, i, j):
    return model.x[i, j] <= model.ev[i, j]


# is ev/se power[i] <= evse[i] max power output for period j?
def se_mpo_constraint_rule(model, i, j):
    return model.x[i, j] <= model.se[i, j]


# the next line creates one constraint for each member of the set model.J
model.EX_MPO_Constraint = Constraint(model.J, rule=demand_constraint_rule)
model.EV_MPI_Constraint = Constraint(model.I, model.J, rule=ev_mpi_constraint_rule)
model.SE_MPO_Constraint = Constraint(model.I, model.J, rule=se_mpo_constraint_rule)


# --------------------------------------------------------------------------
# create a model instance and optimize
# https://pyomo.readthedocs.io/en/stable/working_abstractmodels/instantiating_models.html
# --------------------------------------------------------------------------

tgc = model.create_instance()

opt = pyomo.environ.SolverFactory("cplex")  # or 'mosek' or 'glpk'

opt.solve(tgc)

# --------------------------------------------------------------------------
# display solution
# --------------------------------------------------------------------------

# tgc.pprint()
print("\n")
print(f"Power to be supplied EVSE 1: {[round(tgc.x[1, x].value, 2) for x in range(1, 6)]}")
print(f"Power to be supplied EVSE 2: {[round(tgc.x[2, x].value, 2) for x in range(1, 6)]}")
print(f"Power to be supplied EVSE 3: {[round(tgc.x[3, x].value, 2) for x in range(1, 6)]}")
print("\nObjective Outcome = ", round(value(tgc.OBJ), 4))



Power to be supplied EVSE 1: [5.0, 7.36, 5.0, 3.68, 0.0]
Power to be supplied EVSE 2: [5.0, 7.36, 5.0, 3.68, 0.0]
Power to be supplied EVSE 3: [4.72, 0.0, 4.72, 3.68, 0.0]

Objective Outcome =  0.1389


### 🚧 Pyomo-2

#### Model

The Enexis Grid Connection (ECG) utilization (%) is the highest priority, as modelled in ``pyomo-1``.
 
Next, the customer satisfaction is taken into account as second priority. This will be measured as % of ``desired charge`` $dc_{i}$ which has been delivered at the time the customer leaves. The ``final charge`` $fc_{i}$ in kWh can be calculated when we know for each EV the parking time ($pt_{i}$).

$$
fc_{i} = \displaystyle\sum_{j=1}^{n}  pt_{j}  \; x_{ij}
$$

Please note that the model shouldn't be penalized for unrealistic customer demands. The customers ``desired charge`` $dc_{i}$ in (kWh) may never be higher than the ``realistc charge`` $rc_{i}$, being the maximum energy which can be deliverd at maximum charging power during the time of parking!  

$$
rc_{i} = \sum_{j=1}^{n} pt_{j} \min(ev_{ij}, se_{ij}) \\
$$

The customer satisfaction will be expressed as: 

$$
\text{customer satisfaction}_{i} =  \frac{fc_{i}}{rc_{i}} \; \\\\
$$

$$ \forall i \in \{1, 2, \ldots, m\}, \; \forall j \in \{1, 2, \ldots, n\} $$

##### Objective Function

Modifying the ``pyomo-1`` model results in the following ``Objective function`` to be minimized:  
<br>
<br>

$$
\begin{align*}
\text{minimize} \; &\sum_{j=1}^{n} w_{j} \left( 1 - \frac{\sum_{i=1}^{m} x_{ij}}{ex_{j}} \right) + \sum_{i=1}^{m} \left(1 - \frac{fc_{i}}{rc_{i}}  \right) \\\\
\text{where} \; 
& w_{j} = \frac{pt_{j}}{\sum_{k=1}^{n} pt_{k}}, \quad 
fc_{i} = \sum_{j=1}^{n} pt_{j} x_{ij}, \quad 
rc_{i} = \sum_{j=1}^{n} pt_{j} \min(ev_{ij}, se_{ij}), \\
& \forall j,k \in \{1, 2, \ldots, n\}, \; \forall i \in \{1, 2, \ldots, m\}
\end{align*}
$$

$$
\begin{array}{rllll}
\text{Note: } & \\
& i \text{ represents a specific charge session EV/SE with } m \text{ being the maximum parallel charge sessions possible.} \\
& j \text{ represents a specific time period out of } n \text{ time periods in the forecast horizon.}
\end{array} 
$$

##### Decision Variables

The charging power for EV/SE $i$ at time slot $j$ are the decision variables, $x_{ij}$ 

##### Constraints

The charging power to be used ($x_{ij}$) in kW is subject to 5 constraints:

1. always needs to be positive, Vehicle to Grid (V2G) is **not** allowed.
1. less or equal than the Enexis maximum power output ($ex_j$) of Grid Connection. 
1. less or equal than the maximum power input ($ev_j$) of the EV onboard charger (obc)
1. less or equal than the maximum power output ($se_j$) of the EVSE.
1. the ``final charge`` $fc_{i}$ of EV $i$  should be less or equal to the ``realistic charge`` $rc_{i}$  
(which is less or equal to the ``desired charge`` $dc_{i}$)


$$
\begin{array}{rllll}
\text{s.t.} & x_{ij} \geq 0  &\\
\sum_{i=1}^{m} & x_{ij} \leq ex_{j} &\\
& x_{ij} \leq ev_{ij} &\\
& x_{ij} \leq se_{ij} &\\
\sum_{j=1}^{n} & pt_{j} x_{ij} \leq rc_{i} & 
\end{array} 
$$



$$
\begin{array}{ll}
\forall i \in \{1, 2, \ldots, m,\} & \text{i =  charge session,} & \text{m = 3}  \\
\forall j \in \{1, 2, \ldots, n\} &\text{j =  time period,} & \text{n = 5}
\end{array}
$$

This example relates to the ``pyomo_1`` sheet in this workbook om OneDrive: [TGC_LP](https://1drv.ms/x/s!AiogHeTeve1hj5NFFlBANUYBfoFAcA?e=Kj0bdQ)

#### Excel: results (sheet ``pyomo_2``) 

<img src="./images/pyomo_2.png" width="1000">

#### Code & Results

In [ ]:
# under construction

### 🚧 Pyomo-3

🎅 To nice too have: include energy costs 💰 in weighting factor

#### Model

Pyomo-2 to be extended with the costs for energy by changing the weighting factor


##### Objective Function


##### Decision Variables


##### Constraints



#### Excel: results (sheet ``pyomo_3``) 

<img src="./images/pyomo_3.png" width="1000">

#### Code & Results

In [ ]:
# under construction